# Running an simulaulation model of a queueing system without priorities

The simulation model of a queueing system supports various types of input and service distributions, any number of channels. It is possible to specify the maximum length of the queue. By default, it is an infinite queue. In Kendall's notation: GI/G/n/r and GI/G/n

To run the simulation model, it requires 4 steps:
- creating an instance of the simulation model
- specifying the source distribution
- specifying the service distribution
- running the model


#### Import the QueueingSystemSimulator class and create an instance of the system. 
When creating it, specify the number of channels *n* and the maximum queue length *r*. If not specified, the queue is considered unlimited by default.

In [2]:
from most_queue.sim.base import QsSim

n = 3
r = 100

qs = QsSim(n, buffer=r)

#### Setting the input stream.
The *set_sources* method takes two parameters:
- distribution parameters
- type of distribution

For exponential distribution, you only need to pass one parameter - the intensity of the incoming flow. For most distributions, parameters are specified as a list.

#### Supported types of distributions
| Distribution Type | Type | Parameters |
| --- | --- | --- |
| Exponential | 'M' | mu
| Hyper-exponential 2nd order | 'N' | [y1, mu1, mu2]
| Gamma distribution | 'Gamma' | [mu, alpha]
| Erlang | 'E'| [r, mu]
| Cox 2nd order | 'C' | [y1, mu1, mu2]
| Pareto | 'Pa'| [alpha, K]
| Deterministic | 'D' | [b]
| Uniform | 'Uniform'| [mean, half_interval]
| Normal | 'Norm'| [mean, standard_deviation]

In [3]:
l = 1.0
qs.set_sources(l, 'M')

#### Set the service time distribution. Calculate the service intensity through the utilization coefficient and the number of channels

In [4]:
ro = 0.8  # коэффициент загрузки
mu = l / (ro * n)  # интенсивность обслуживания
qs.set_servers(mu, 'M')

#### To run the simulation, you need to call the *run* method and pass the number of jobs

In [5]:
num_of_jobs = 1000000
qs.run(num_of_jobs)

Start simulation


Job served:    | 0/100 [00:00<?, ?it/s]10000/1000000:   1%|          | 1/100 [00:00<00:09, 10.84it/s]10000/1000000:   2%|▏         | 2/100 [00:00<00:09, 10.56it/s]20000/1000000:   2%|▏         | 2/100 [00:00<00:09, 10.56it/s]30000/1000000:   3%|▎         | 3/100 [00:00<00:09, 10.56it/s]30000/1000000:   4%|▍         | 4/100 [00:00<00:09, 10.32it/s]40000/1000000:   4%|▍         | 4/100 [00:00<00:09, 10.32it/s]50000/1000000:   5%|▌         | 5/100 [00:00<00:09, 10.32it/s]50000/1000000:   6%|▌         | 6/100 [00:00<00:09, 10.16it/s]60000/1000000:   6%|▌         | 6/100 [00:00<00:09, 10.16it/s]70000/1000000:   7%|▋         | 7/100 [00:00<00:09, 10.16it/s]70000/1000000:   8%|▊         | 8/100 [00:00<00:09, 10.22it/s]80000/1000000:   8%|▊         | 8/100 [00:00<00:09, 10.22it/s]90000/1000000:   9%|▉         | 9/100 [00:00<00:08, 10.22it/s]90000/1000000:  10%|█         | 10/100 [00:00<00:08, 10.02it/s]100000/1000000:  10%|█         | 10/100 [00:00<00:08, 10.02it/s]110000/1000000:  11%|█      

Simulation is finished



#### After completing the simsimulation, you can obtain the initial moments of waiting and staying times, as well as the probabilities of system states.

In [6]:
print("Time spent ", qs.time_spent)
v_sim = qs.v
w_sim = qs.w
p_sim = qs.p
print(v_sim)

Time spent  10.882635559999999
[4.997995299386078, 44.99256956507027, 576.7893492504401]


#### Compare the results of simuimulation with the results of numerical calculation for the model M/M/n/r

In [7]:
from most_queue.theory.fifo.mmnr import MMnrCalc
from most_queue.general.tables import times_print, probs_print

mmnr = MMnrCalc(l, mu, n, r)
w = mmnr.get_w()
times_print(w_sim, w)


Initial moments of waiting time in the system
---------------------------------------------
       #       |      Num      |      Sim      
---------------------------------------------
       1       |    2.5888     |    2.5983     
       2       |     20.71     |    21.001     
       3       |    248.52     |    252.49     



#### Check for correctness of simulation results with the results of calculation for the model M/D/n

In [8]:
from most_queue.theory.fifo.m_d_n import MDn

qs = QsSim(n)

qs.set_sources(l, 'M')
qs.set_servers(1.0 / mu, 'D')

qs.run(num_of_jobs)

mdn = MDn(l, 1 / mu, n)
p_num = mdn.calc_p()
p_sim = qs.get_p()

print("Time spent ", qs.time_spent)

probs_print(p_sim, p_num, 10)

Start simulation


Job served:    | 0/100 [00:00<?, ?it/s]10000/1000000:   1%|          | 1/100 [00:00<00:08, 11.98it/s]10000/1000000:   2%|▏         | 2/100 [00:00<00:08, 10.93it/s]20000/1000000:   2%|▏         | 2/100 [00:00<00:08, 10.93it/s]30000/1000000:   3%|▎         | 3/100 [00:00<00:08, 10.93it/s]30000/1000000:   4%|▍         | 4/100 [00:00<00:08, 10.68it/s]40000/1000000:   4%|▍         | 4/100 [00:00<00:08, 10.68it/s]50000/1000000:   5%|▌         | 5/100 [00:00<00:08, 10.68it/s]50000/1000000:   6%|▌         | 6/100 [00:00<00:08, 10.72it/s]60000/1000000:   6%|▌         | 6/100 [00:00<00:08, 10.72it/s]70000/1000000:   7%|▋         | 7/100 [00:00<00:08, 10.72it/s]70000/1000000:   8%|▊         | 8/100 [00:00<00:08, 10.46it/s]80000/1000000:   8%|▊         | 8/100 [00:00<00:08, 10.46it/s]90000/1000000:   9%|▉         | 9/100 [00:00<00:08, 10.46it/s]90000/1000000:  10%|█         | 10/100 [00:00<00:08, 10.17it/s]100000/1000000:  10%|█         | 10/100 [00:00<00:08, 10.17it/s]110000/1000000:  11%|█      

Simulation is finished

Time spent  10.000411461999999
------------------------------------
      Probabilities of states       
------------------------------------
 #  |      Num      |      Sim      
------------------------------------
 0  |   0.049841    |   0.049587    
 1  |    0.13286    |    0.13168    
 2  |    0.18476    |    0.18396    
 3  |    0.18195    |    0.18156    
 4  |    0.14595    |    0.14594    
 5  |    0.10403    |    0.10471    
 6  |   0.069852    |   0.070627    
 7  |   0.045764    |   0.046252    
 8  |   0.029764    |   0.030001    
 9  |   0.019336    |    0.0196     
------------------------------------

